In [1]:
import fnmatch
import os
import random
import re
import threading
import pandas as pd
import time
import tqdm

import librosa
import numpy as np
import torch

from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen

import matplotlib.pyplot as plt

%matplotlib inline

FILE_PATTERN = r'*.wav'

# Load data

In [2]:
def wavenet_encode(audio):
    
    # Load the model weights.
    checkpoint_path = './wavenet-ckpt/model.ckpt-200000'
    
    # Pass the audio through the first half of the autoencoder,
    # to get a list of latent variables that describe the sound.
    # Note that it would be quicker to pass a batch of audio
    # to fastgen. 
    encoding = fastgen.encode(audio, checkpoint_path, len(audio))
    
    # Reshape to a single sound.
    return encoding.reshape((-1, 16))

def randomize_files(files):
    '''Shuffle loaded files'''
    for file in files:
        file_index = random.randint(0, (len(files) - 1))
        yield files[file_index]


def find_files(directory, pattern='*.wav'):
    '''Recursively finds all files matching the pattern.'''
    files = []
    fnames = []
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(os.path.join(root, filename))
            fnames.append(filename)
    return files, fnames

def get_category(fname):
    '''Parse type from fnamepar'''
    return fname.split('/')[-1].split('_')[0]

def load_generic_audio(files, sample_rate, amount):
    '''Generator that yields audio waveforms from the directory.'''
    for it, filename in enumerate(files):
        if it == amount:
            break
        category_id = get_category(filename)
        audio, _ = librosa.load(filename, sr=sample_rate, mono=True)
        audio = audio.reshape(-1, 1)
        yield audio, filename, category_id


def trim_silence(audio, threshold, frame_length=512):
    '''Removes silence at the beginning and end of a sample.'''
    if audio.size < frame_length:
        frame_length = audio.size
    energy = librosa.feature.rmse(audio, frame_length=frame_length)
    frames = np.nonzero(energy > threshold)
    indices = librosa.core.frames_to_samples(frames)[1]

    # Note: indices can be an empty array, if the whole audio was silence.
    return audio[indices[0]:indices[-1]] if indices.size else audio[0:0]



class AudioReader(object):
    '''Generic background audio reader that preprocesses audio files
    and add tham in lists'''

    def __init__(self,
                 audio_dir,
                 sample_rate,
                 silence_threshold=None,
                 sample_size=None,
                 load_size=None):
        self.audio_dir = audio_dir
        self.sample_rate = sample_rate
        self.sample_size = sample_size
        self.silence_threshold = silence_threshold
        self.load_size = load_size
        self.counter = 0
        self.time = time.time()

        # TODO Find a better way to check this.
        # Checking inside the AudioReader's thread makes it hard to terminate
        # the execution of the script, so we do it in the constructor for now.

        self.files, self.fnames = find_files(audio_dir)
        if not self.files:
            raise ValueError("No audio files found in '{}'.".format(audio_dir))
        self.pred_category = np.full(len(self.files), True)
        if load_size is not None:
            self.data = [0]*load_size
            self.id = [0]*load_size
        else:    
            self.data = [0]*len(self.files)
            self.id = [0]*len(self.files)
        # Determine the number of mutually-exclusive categories we will
        # accomodate in our embedding table.

    def read(self):
        #Read dataset
        
        iterator = load_generic_audio(self.files, self.sample_rate, self.load_size)
        for audio, filename, category_id in iterator:
            if self.silence_threshold is not None:
                # Remove silence
                audio = trim_silence(audio[:, 0], self.silence_threshold)
                audio = audio.reshape(-1)
                if audio.size == 0:
                    self.pred_category[self.counter]=False


            self.data[self.counter] = audio
            self.id[self.counter] = category_id
            self.counter += 1
            if self.counter % 400 == 0:
                print (time.time() - self.time,self.counter)
        return self

In [9]:
audio_dir = './data_v_7_stc/audio' 
sr = 16000
silence_threshold = 8e-4
audio_reader = AudioReader(audio_dir,sr,silence_threshold=silence_threshold)
audio_reader.read()

KeyboardInterrupt: 

In [18]:
n=len(audio_reader.data)
start = time.time()
for i,arr in enumerate(audio_reader.data):
    if arr.size == 0:
        continue
    wn = wavenet_encode(arr)
    np.save("wavenet_features/" + audio_reader.fnames[i], wn)
    if i%500 == 0:

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
0 5.386393308639526
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

...

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
3000 16440.397745609283
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
I

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

KeyboardInterrupt: 

In [11]:
test_dir='./data_v_7_stc/test'
test_reader = AudioReader(test_dir,sr,silence_threshold=silence_threshold)
test_reader.read()
n=len(test_reader.data)
start = time.time()
for i,arr in enumerate(test_reader.data):
    if arr.size == 0:
        continue
    wn = wavenet_encode(arr)
    np.save("test_feats/" + test_reader.fnames[i], wn)
    if i%300 == 0:
        print (i, time.time()-start)

2.7758562564849854 400
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
0 9.799367189407349
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Restoring